### 행정안전부 보도자료 데이터 수집
- https://mois.go.kr
- post 방식 사용 : url에 정보가있는것이아닌, 딕셔너리로 만들어서,함수로 만들어서, 아규먼트로 설정해서 실행해서 가져옴
- 뉴스소식 -> 보도자료

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url = 'https://mois.go.kr/frt/bbs/type010/commonSelectBoardList.do?bbsId=BBSMSTR_000000000008'

In [6]:
# post방식이기에 params를 같이 실어서 요청 
# 왜? 정적페이지 인데, url이 바뀌진 않음, 
page = 1
params = {
    'nttId': 0,
    'bbsTyCode': 'BBST03',
    'bbsAttrbCode': 'BBSA03',
    'authFlag': 'Y',
    'pageIndex': page,
    'cal_url': ' /sym/cal/EgovNormalCalPopup.do',
    'searchCnd': 0,
}
params

{'nttId': 0,
 'bbsTyCode': 'BBST03',
 'bbsAttrbCode': 'BBSA03',
 'authFlag': 'Y',
 'pageIndex': 1,
 'cal_url': ' /sym/cal/EgovNormalCalPopup.do',
 'searchCnd': 0}

In [11]:
response = requests.post(url,params)
response

<Response [200]>

In [13]:
dom = BeautifulSoup(response.content, 'html.parser')
dom


<!DOCTYPE html>

<html lang="ko">
<head>
<title>
행정안전부&gt;  뉴스·소식&gt; 보도자료&gt; 보도자료
</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width,maximum-scale=1,minimum-scale=1,user-scalable=no" name="viewport"/>
<meta content="행정안전부" name="author"/>
<meta content="행정안전부" name="subject"/>
<meta content="행정안전부, 행안부, 정부혁신, 지방자치, 생활자치, 규제개혁, 지역경제 활성화, 국민생활 불편해소, 지방재정 개혁, 행정관리, 디지털 정부, 복지서비스, 복지사각지대, 지방공기업 경쟁력, 국가상징, 국가상훈 관리, 행정감사제도, 국무회의 서무, 법령 및 조약 공포, 정부조직과 정원, 행정능률, 전자정부, 개인정보보호, 정부청사 관리, 낙후지역 지원, 지방자치단체 분쟁조정, 선거 국민투표 지원 사무, 국가 행정사무, 기획조정실, 정부혁신조직실, 전자정부국, 지방자치분권실, 지방재정경제실, 장차관직속기관, 지방자치인재개발원, 국가기록원, 정부청사관리본부, 국가정보자원관리원, 이북5도, 국가민방위재난안전연구원, 국립과학수사연구원, 경찰청" name="keywords"/>
<meta content="행정안전부 홈페이지에 오신것을 환영합니다." property="og:description"/>
<meta content="" lang="ko" name="description"/>
<meta content="all" name="robots"/>
<link href="/images/frt/common/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/imag

In [18]:
# 가져오고싶은 테이블의 element를 확인, table_style1 클래스 아래, tbody 아래 tr 엘리먼트가져옴
elements = dom.select('.table_style1 > tbody > tr')
# 갯수가 맍으면 맞음 (10개) , 객체가 리스트로 저정됨
len(elements)

10

In [30]:
# elements로 가져온 데이터를 구별해주는것.
# 개발자도구로 해당 element를 좀 고민해봐야함.
datas = []
for element in elements:
    datas.append({
    # nth-child()를 사용하여 td밑에 있는 데이터를 가져옴
    'no': element.select_one('td:nth-child(1)').text.strip(),
    'title': element.select_one('td:nth-child(2)').text.strip(),
   #'link': element.select_one('td:nth-child(2) > div > a').get('href'),
    'writer': element.select_one('td:nth-child(4)').text.strip(),
    'date': element.select_one('td:nth-child(5)').text.strip(),
    'pv': element.select_one('td:nth-child(6)').text.strip(),   
    })
mois_df = pd.DataFrame(datas)
mois_df

,no,title,writer,date,pv
0,10620,위기경보‘심각’발령에 따른 코로나19 긴급대책비 지원,재난관리정책과,2020.02.26.,439
1,10619,"터널 안 교통사고, 최근 5년간 3,218건, 인명피해 7,472명",예방안전과,2020.02.25.,310
2,10618,‘제7기(2014~2018) 지방의회백서’ 발간,선거의회과,2020.02.25.,134
3,10617,"정부청사, 코로나19 유입대비 출입강화, 집중방역 실시",노사후생과,2020.02.24.,371
4,10616,"소하천 토지변경 없으면, 점용허가 끝나도 원상복구 면제",재난경감과,2020.02.24.,287
5,10615,"각종 규제에 가로막힌 접경지역 발전에 올해 2,160억원 투자",지역균형발전과,2020.02.24.,382
6,10614,"행안부, 대한적십자사와 손잡고 일원화된 재난심리회복 체계 구축",재난구호과,2020.02.23.,420
7,10613,"행안부, 코로나19 확산에 따른 현장 대비상황 종합 점검",안전기획과,2020.02.22.,262
8,10612,"진영 장관, 코로나19 확산 방지에 전국 지자체 총력 대응 요청",자치행정과,2020.02.20.,2410
9,10611,캠핑장 안전사고의 30.8%가 화상‧중독사고,예방안전과,2020.02.20.,468
